<a href="https://colab.research.google.com/github/custerc/miscellaneous-marketing-code/blob/main/Blog_Traffic_Drop_Analysis_for_Cockroach_Labs_V2_(GA4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import csv

#create empty lists for data from the current and previous periods in the export
current_list = []
previous_list = []

#open the csv file
with open (str('data-export.csv'), 'r') as file:
  data_reader=csv.reader(file)
  datalist = list(data_reader)

# Flag to determine whether to add rows to 'current' or 'previous'
add_to_current = True

for row in datalist[9:]:
# handle the empty rows and second round of dates
  if not row:
    continue
  if 'Start date' in row[0]:
    continue
  if 'End date' in row[0]:
    continue

  # keep add_to_current true until you hit # All Users
  if row[0] == '# All Users':
    add_to_current = False
    continue  # Skip the row containing '# All Users'

  # appending to the appropriate lists
  if add_to_current:
    current_list.append(row)
  else:
    previous_list.append(row)


In [6]:
# create two dataframes with the two lists
df_current = pd.DataFrame(current_list,columns=['Landing page', 'Sessions', 'Users', 'New users', 'Average engagement time per session', 'Conversions', 'Total revenue'])
df_previous = pd.DataFrame(previous_list,columns=['Landing page', 'Sessions_prev', 'Users_prev', 'New users_prev', 'Average engagement time per session_prev', 'Conversions_prev', 'Total revenue_prev'])

# merge the dataframes and clean the data
joined = pd.merge(df_current, df_previous, on='Landing page', how='outer')
joined.replace(',','', regex=True, inplace=True)
joined.replace('%','', regex=True, inplace=True)
joined = joined.drop(index=0)

# convert to numeric datatypes
joined[['Sessions', 'Users', 'New users', 'Average engagement time per session', 'Conversions', 'Total revenue', 'Sessions_prev', 'Users_prev', 'New users_prev', 'Average engagement time per session_prev', 'Conversions_prev', 'Total revenue_prev']] = joined[['Sessions', 'Users', 'New users', 'Average engagement time per session', 'Conversions', 'Total revenue', 'Sessions_prev', 'Users_prev', 'New users_prev', 'Average engagement time per session_prev', 'Conversions_prev', 'Total revenue_prev']].apply(pd.to_numeric)

# create new columns tracking the percentage and raw change in sessions in the joined df
joined['SessionsDelta_perc'] = (joined['Sessions'] - joined['Sessions_prev']) / joined['Sessions_prev'] * 100
joined['SessionsDelta_raw'] = (joined['Sessions'] - joined['Sessions_prev'])
joined = joined.round({'SessionsDelta_perc': 1})


joined.dtypes


Landing page                                 object
Sessions                                      int64
Users                                         int64
New users                                     int64
Average engagement time per session         float64
Conversions                                   int64
Total revenue                                 int64
Sessions_prev                                 int64
Users_prev                                    int64
New users_prev                                int64
Average engagement time per session_prev    float64
Conversions_prev                              int64
Total revenue_prev                            int64
SessionsDelta_perc                          float64
SessionsDelta_raw                             int64
dtype: object

In [8]:
# CREATE THE REPORT TO OUTPUT - WORK IN PROGRESS
# TODO
  # Add more columns to output
  # Add directions and unify to single code box
  # charts?
  # output session dates as a reminder?

# Set minimum number of sessions required (in both current and previous periods) to return data in chart
session_floor = 50
# Set number of rows to output
rows_to_output = 2
# set column to sort by. Probably you want 'SessionDelta_perc' or 'SessionsDelta_raw'
sortbycol = 'SessionsDelta_perc'

output = joined.sort_values(by=[sortbycol], ascending=True)
output = output.drop(output[output.Sessions_prev < session_floor].index)
output = output.drop(output[output.Sessions < session_floor].index)
output = output[['Landing page', 'SessionsDelta_perc', 'SessionsDelta_raw', 'Sessions', 'Sessions_prev', 'Average engagement time per session', 'Average engagement time per session_prev']]

output.head(rows_to_output)

,Landing page,SessionsDelta_perc,SessionsDelta_raw,Sessions,Sessions_prev,Average engagement time per session,Average engagement time per session_prev
104,/blog/serverless-free,-71.3,-194,78,272,87.256410,50.731618
90,/blog/5-fintech-companies-refused-to-sacrifice...,-63.9,-163,92,255,21.097826,15.423529
